In [ ]:
import pandas as pd
from gurobipy import *

#input parameters
FILE_PATH = "jeroen_punt_nl_dynamische_stroomprijzen_jaar_2024.csv"
FIXED_PRICE = 0.318  # €/kWh (fixed contract incl. taxes)
POWER_KW = 7.4       # charging power (1-phase wallbox)
CHARGE_START = 18    # start hour (18:00)
CHARGE_END = 7       # end hour next morning (07:00)
TAX_SURCHARGE = 0.10880
BTW = 1.21

#data set
df = pd.read_csv(FILE_PATH, sep=';', decimal=',')
df['datum_nl'] = pd.to_datetime(df['datum_nl'])
df['hour'] = df['datum_nl'].dt.hour
df['weekday'] = df['datum_nl'].dt.day_name()

#include taxes in price
df['prijs_incl_belastingen'] = (df['prijs_excl_belastingen'] + TAX_SURCHARGE) * BTW


#calculating costs for different scenarios
def calculate_scenario(name, active_days, energy_kwh_per_day):
    # setting charging window
    df_charge = df[(df['hour'] >= CHARGE_START) | (df['hour'] < CHARGE_END)].copy()

    # selecting the days
    if active_days:
        df_charge = df_charge[df_charge['weekday'].isin(active_days)]

    # immediate charging (first available hours that day)
    def get_first_hours(group):
        needed_hours = int(energy_kwh_per_day / POWER_KW)
        return group.head(needed_hours)

    df_nonopt = (
        df_charge.groupby(df_charge['datum_nl'].dt.date, group_keys=False)
        .apply(get_first_hours)
    )

    df_nonopt['cost_dynamic'] = df_nonopt['prijs_incl_belastingen'] * POWER_KW
    df_nonopt['cost_fixed'] = FIXED_PRICE * POWER_KW

    daily_costs = (
        df_nonopt.groupby(df_nonopt['datum_nl'].dt.date)[['cost_dynamic', 'cost_fixed']].sum()
    )
    total_dynamic = daily_costs['cost_dynamic'].sum()
    total_fixed = daily_costs['cost_fixed'].sum()

    # === OPTIMIZED (Gurobi): choose cheapest hours between 18:00 and 07:00 ===
    unique_dates = sorted(df_charge['datum_nl'].dt.date.unique())
    optimized_daily_costs = []

    for i in range(len(unique_dates) - 1):
        day_today = unique_dates[i]
        next_day = unique_dates[i + 1]

        # Select the 18:00–07:00 window (spanning two dates)
        window = df[(df['datum_nl'].dt.date == day_today) & (df['hour'] >= CHARGE_START)]
        window = pd.concat([window, df[(df['datum_nl'].dt.date == next_day) & (df['hour'] < CHARGE_END)]])
        window = window.sort_values('datum_nl')

        if window.empty:
            continue

        timesteps = range(len(window))
        prices = {t: window.iloc[t]['prijs_incl_belastingen'] for t in timesteps}

        model = Model()
        charge = model.addVars(timesteps, lb=0, ub=POWER_KW, name="charge")

        # Energy requirement constraint
        model.addConstr(sum(charge[t] for t in timesteps) == energy_kwh_per_day)

        # Objective: minimize cost
        model.setObjective(sum(prices[t] * charge[t] for t in timesteps), GRB.MINIMIZE)
        model.setParam('OutputFlag', 0)
        model.optimize()

        optimized_daily_costs.append(model.objVal)

    total_optimized = sum(optimized_daily_costs)
    diff_opt_vs_fixed = total_optimized - total_fixed
    diff_opt_vs_dyn = total_optimized - total_dynamic

    return {
        "Scenario": name,
        "Active days": ", ".join(active_days) if active_days else "Every day",
        "Energy/day (kWh)": energy_kwh_per_day,
        "Total dynamic (€)": total_dynamic,
        "Total fixed (€)": total_fixed,
        "Total optimized (€)": total_optimized,
        "Δ opt - fixed (€)": diff_opt_vs_fixed,
        "Δ opt - dyn (€)": diff_opt_vs_dyn
    }


# === Define all four scenarios ===
scenarios = [
    ("Distant commuter", [], 29.6),                      # daily
    ("City hopper", [], 7.4),                            # daily
    ("Road tripper", ["Saturday", "Sunday"], 29.6),      # weekends
    ("Grocery grabber", ["Wednesday", "Sunday"], 7.4)    # twice per week
]

# === Run all scenarios ===
results = [calculate_scenario(*s) for s in scenarios]
df_results = pd.DataFrame(results)

df_results.to_csv("scenario_costs_2024.csv", index=False)
print("Results saved to 'scenario_costs_2024.csv'")

# === Final results table (no pandas formatting, no lambda) ===

# Use ASCII-only column titles to avoid terminal/encoding issues
print(f"{'Scenario':<20} {'Active days':<25} {'Energy/day (kWh)':>17} "
      f"{'Total dynamic (€)':>20} {'Total fixed (€)':>20} {'Total optimized (€)':>23} "
      f"{'Difference optimized  (fixed) (€)':>20} {'Difference optimized  (dynamic) (€)':>20}")
print("-" * 165)

# Print directly from the 'results' list to avoid column-name mismatches
for r in results:
    print(f"{r['Scenario']:<20} {r['Active days']:<25} {r['Energy/day (kWh)']:>17.1f} "
          f"{r['Total dynamic (€)']:>20.2f} {r['Total fixed (€)']:>20.2f} "
          f"{r['Total optimized (€)']:>23.2f} {r['Δ opt - fixed (€)']:>20.2f} {r['Δ opt - dyn (€)']:>20.2f}")



df.head()


Results saved to 'scenario_costs_2024.csv'
Scenario             Active days                Energy/day (kWh)    Total dynamic (€)      Total fixed (€)     Total optimized (€) Difference optimized  (fixed) (€) Difference optimized  (dynamic) (€)
---------------------------------------------------------------------------------------------------------------------------------------------------------------------
Distant commuter     Every day                              29.6              2337.97              3445.08                 2256.47             -1188.62               -81.50
City hopper          Every day                               7.4               605.68               861.27                  550.45              -310.82               -55.23
Road tripper         Saturday, Sunday                       29.6               665.34               978.93                  610.20              -368.73               -55.14
Grocery grabber      Wednesday, Sunday                       7.4       

,datum_nl,datum_utc,prijs_excl_belastingen,hour,weekday,prijs_incl_belastingen
0,2024-01-01 00:00:00,2023-12-31 23:00:00,0.00,0,Monday,0.13
1,2024-01-01 01:00:00,2024-01-01 00:00:00,0.00,1,Monday,0.13
2,2024-01-01 02:00:00,2024-01-01 01:00:00,0.00,2,Monday,0.13
3,2024-01-01 03:00:00,2024-01-01 02:00:00,-0.00,3,Monday,0.13
4,2024-01-01 04:00:00,2024-01-01 03:00:00,-0.00,4,Monday,0.13


In [12]:
df.tail()

,datum_nl,datum_utc,prijs_excl_belastingen,hour,weekday,prijs_incl_belastingen
8778,2024-12-31 19:00:00,2024-12-31 18:00:00,0.07,19,Tuesday,0.22
8779,2024-12-31 20:00:00,2024-12-31 19:00:00,0.06,20,Tuesday,0.20
8780,2024-12-31 21:00:00,2024-12-31 20:00:00,0.02,21,Tuesday,0.15
8781,2024-12-31 22:00:00,2024-12-31 21:00:00,0.01,22,Tuesday,0.14
8782,2024-12-31 23:00:00,2024-12-31 22:00:00,0.00,23,Tuesday,0.13


In [13]:
import plotly.graph_objects as go
from random import choice

# === Pick one random day from the dataset ===
unique_dates = sorted(df['datum_nl'].dt.date.unique())
random_day = choice(unique_dates[:-1])                # random day except the last
next_day = unique_dates[unique_dates.index(random_day) + 1]

# === Extract the 18:00–07:00 window for that night ===
window = df[(df['datum_nl'].dt.date == random_day) & (df['hour'] >= CHARGE_START)]
window = pd.concat([
    window,
    df[(df['datum_nl'].dt.date == next_day) & (df['hour'] < CHARGE_END)]
])
window = window.sort_values('datum_nl').reset_index(drop=True)

# === Non-optimized: charge immediately at 18:00 until full ===
energy_needed = 29.6                      # kWh for Distant Commuter
needed_hours = int(energy_needed / POWER_KW)   # 29.6 / 7.4 = 4 hours
nonopt_hours = window.head(needed_hours)

# === Optimized: choose cheapest hours with Gurobi ===
timesteps = range(len(window))
prices = {t: window.iloc[t]['prijs_incl_belastingen'] for t in timesteps}

model = Model()
charge = model.addVars(timesteps, lb=0, ub=POWER_KW)
model.addConstr(sum(charge[t] for t in timesteps) == energy_needed)
model.setObjective(sum(prices[t] * charge[t] for t in timesteps), GRB.MINIMIZE)
model.setParam('OutputFlag', 0)
model.optimize()
optimized_schedule = [charge[t].X for t in timesteps]

# === Build Plotly figure ===
fig = go.Figure()

# Price line
fig.add_trace(go.Scatter(
    x=window['datum_nl'], y=window['prijs_incl_belastingen'],
    mode='lines', name='Dynamic price (€/kWh)',
    line=dict(color='black', width=1.6),
    yaxis='y2'
))

# Non-optimized charging bars (immediate at 18:00)
fig.add_trace(go.Bar(
    x=nonopt_hours['datum_nl'],
    y=[POWER_KW] * len(nonopt_hours),
    name='Non-optimized (18:00 start)',
    marker_color='red',
    opacity=0.5,
    yaxis='y1'
))

# Optimized charging bars
fig.add_trace(go.Bar(
    x=window['datum_nl'],
    y=optimized_schedule,
    name='Optimized (cheapest hours)',
    marker_color='royalblue',
    opacity=0.7,
    yaxis='y1'
))

# Layout
fig.update_layout(
    title=f"Distant Commuter – Optimized vs. Immediate Charging ({random_day})",
    template="plotly_white",
    hovermode="x unified",
    xaxis=dict(title="Time (18:00 – 07:00 next day)"),
    yaxis=dict(title="Charging Power (kW)", range=[0, POWER_KW + 1]),
    yaxis2=dict(title="Price (€/kWh)", overlaying="y", side="right", tickprefix="€"),
    legend=dict(orientation="h", yanchor="top", y=-0.25, xanchor="center", x=0.5)
)

fig.show()


In [15]:
import pandas as pd
from gurobipy import *

# === INPUT PARAMETERS ===
FILE_PATH = "jeroen_punt_nl_dynamische_stroomprijzen_jaar_2024.csv"
FIXED_PRICE = 0.318  # €/kWh (fixed contract incl. taxes)
POWER_KW = 7.4       # charging power (1-phase wallbox)
CHARGE_START = 18    # start hour (18:00)
CHARGE_END = 7       # end hour next morning (07:00)
TAX_SURCHARGE = 0.10880
BTW = 1.21

# === LOAD DATA ===
df = pd.read_csv(FILE_PATH, sep=';', decimal=',')
df['datum_nl'] = pd.to_datetime(df['datum_nl'])
df['hour'] = df['datum_nl'].dt.hour
df['weekday'] = df['datum_nl'].dt.day_name()

# === INCLUDE TAXES ===
df['prijs_incl_belastingen'] = (df['prijs_excl_belastingen'] + TAX_SURCHARGE) * BTW


# === FUNCTION: CALCULATE ONE SCENARIO ===
def calculate_scenario(name, active_days, energy_kwh_per_day, night_window=False):
    """
    Calculates yearly cost for one scenario:
    - if night_window=True, charges always between 01:00–06:00 at variable prices
    - else: compares immediate (non-opt) vs optimized (Gurobi) charging between 18:00–07:00
    """

    # === CASE 1: NIGHT WINDOW SCENARIO (01:00–06:00 FIXED HOURS) ===
    if night_window:
        df_night = df[(df['hour'] >= 1) & (df['hour'] < 4)].copy()
        if active_days:
            df_night = df_night[df_night['weekday'].isin(active_days)]

        n_hours = len(df_night['hour'].unique())
        energy_per_hour = energy_kwh_per_day / n_hours

        df_night['cost_dynamic'] = df_night['prijs_incl_belastingen'] * energy_per_hour
        total_dynamic = df_night['cost_dynamic'].sum()

        return {
            "Scenario": name,
            "Active days": ", ".join(active_days) if active_days else "Every day",
            "Energy/day (kWh)": energy_kwh_per_day,
            "Total dynamic (€)": total_dynamic,
            "Total fixed (€)": None,
            "Total optimized (€)": None,
            "Δ opt - fixed (€)": None,
            "Δ opt - dyn (€)": None
        }

    # === CASE 2: REGULAR SCENARIOS (18:00–07:00 WINDOW) ===
    df_charge = df[(df['hour'] >= CHARGE_START) | (df['hour'] < CHARGE_END)].copy()

    if active_days:
        df_charge = df_charge[df_charge['weekday'].isin(active_days)]

    # === Non-optimized: charge immediately from 18:00 ===
    def get_first_hours(group):
        needed_hours = int(energy_kwh_per_day / POWER_KW)
        return group.head(needed_hours)

    df_nonopt = (
        df_charge.groupby(df_charge['datum_nl'].dt.date, group_keys=False)
        .apply(get_first_hours)
    )

    df_nonopt['cost_dynamic'] = df_nonopt['prijs_incl_belastingen'] * POWER_KW
    df_nonopt['cost_fixed'] = FIXED_PRICE * POWER_KW

    daily_costs = (
        df_nonopt.groupby(df_nonopt['datum_nl'].dt.date)[['cost_dynamic', 'cost_fixed']].sum()
    )

    total_dynamic = daily_costs['cost_dynamic'].sum()
    total_fixed = daily_costs['cost_fixed'].sum()

    # === Optimized (Gurobi): choose cheapest hours 18:00–07:00 ===
    unique_dates = sorted(df_charge['datum_nl'].dt.date.unique())
    optimized_daily_costs = []

    for i in range(len(unique_dates) - 1):
        day_today = unique_dates[i]
        next_day = unique_dates[i + 1]

        # Select 18:00–07:00 window
        window = df[(df['datum_nl'].dt.date == day_today) & (df['hour'] >= CHARGE_START)]
        window = pd.concat([window, df[(df['datum_nl'].dt.date == next_day) & (df['hour'] < CHARGE_END)]])
        window = window.sort_values('datum_nl')

        if window.empty:
            continue

        timesteps = range(len(window))
        prices = {t: window.iloc[t]['prijs_incl_belastingen'] for t in timesteps}

        model = Model()
        charge = model.addVars(timesteps, lb=0, ub=POWER_KW, name="charge")
        model.addConstr(sum(charge[t] for t in timesteps) == energy_kwh_per_day)
        model.setObjective(sum(prices[t] * charge[t] for t in timesteps), GRB.MINIMIZE)
        model.setParam('OutputFlag', 0)
        model.optimize()

        optimized_daily_costs.append(model.objVal)

    total_optimized = sum(optimized_daily_costs)
    diff_opt_vs_fixed = total_optimized - total_fixed
    diff_opt_vs_dyn = total_optimized - total_dynamic

    return {
        "Scenario": name,
        "Active days": ", ".join(active_days) if active_days else "Every day",
        "Energy/day (kWh)": energy_kwh_per_day,
        "Total dynamic (€)": total_dynamic,
        "Total fixed (€)": total_fixed,
        "Total optimized (€)": total_optimized,
        "Δ opt - fixed (€)": diff_opt_vs_fixed,
        "Δ opt - dyn (€)": diff_opt_vs_dyn
    }


# === DEFINE ALL FIVE SCENARIOS ===
scenarios = [
    ("Distant commuter", [], 29.6, False),                      # daily
    ("City hopper", [], 7.4, False),                            # daily
    ("Road tripper", ["Saturday", "Sunday"], 29.6, False),      # weekends
    ("Grocery grabber", ["Wednesday", "Sunday"], 7.4, False),   # twice per week
    ("Night charger (01-06)", [], 29.6, True)                   # new scenario (variable contract)
]

# === RUN ALL SCENARIOS ===
results = [calculate_scenario(*s) for s in scenarios]
df_results = pd.DataFrame(results)

# === SAVE RESULTS ===
df_results.to_csv("scenario_costs_2024.csv", index=False)
print("Results saved to 'scenario_costs_2024.csv'")

# === PRINT TABLE ===
print(f"\n{'Scenario':<28} {'Active days':<25} {'Energy/day (kWh)':>17} "
      f"{'Total dynamic (€)':>20} {'Total fixed (€)':>20} {'Total optimized (€)':>23} "
      f"{'Δ opt - fixed (€)':>20} {'Δ opt - dyn (€)':>20}")
print("-" * 180)

for r in results:
    print(f"{r['Scenario']:<28} {r['Active days']:<25} {r['Energy/day (kWh)']:>17.1f} "
          f"{(r['Total dynamic (€)'] or 0):>20.2f} {(r['Total fixed (€)'] or 0):>20.2f} "
          f"{(r['Total optimized (€)'] or 0):>23.2f} {(r['Δ opt - fixed (€)'] or 0):>20.2f} {(r['Δ opt - dyn (€)'] or 0):>20.2f}")

print("\n==============================================================\n")


Results saved to 'scenario_costs_2024.csv'

Scenario                     Active days                Energy/day (kWh)    Total dynamic (€)      Total fixed (€)     Total optimized (€)    Δ opt - fixed (€)      Δ opt - dyn (€)
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Distant commuter             Every day                              29.6              2337.97              3445.08                 2256.47             -1188.62               -81.50
City hopper                  Every day                               7.4               605.68               861.27                  550.45              -310.82               -55.23
Road tripper                 Saturday, Sunday                       29.6               665.34               978.93                  610.20              -368.73               -55.14
Grocery grabber              Wednesday, Sunday     